# Data Visualization with GeoPy, Folium and hvplot

**GeoPy**: a Python client to locate the coordinates of addresses, cities, countries, and landmarks across the globe using third-party geocoders and other data sources. https://geopy.readthedocs.io/en/stable/
 
**Folium**: to visualize data that’s been manipulated in Python on an interactive leaflet map. http://python-visualization.github.io/folium/

**hvPlot**: a high-level plotting API built on HoloViews that provides a general and consistent API to create interactively explorable Bokeh plots with panning, zooming, hovering, and clickable/selectable legends. https://hvplot.holoviz.org/

In [23]:
import numpy as np
import pandas as pd
import xarray as xr
import intake
from ipywidgets import widgets
from geopy.geocoders import Nominatim
import folium 
import hvplot.pandas
import warnings
warnings.filterwarnings("ignore")
from os.path import expanduser
home = expanduser("~")

### Choose a location
<a id='selection'></a>

In [24]:
place_box = widgets.Text(description="Enter place:", value="Prague")
display(place_box)

Text(value='Prague', description='Enter place:')

In [25]:
# The Nominatim module is used to retrieve the geographical coordinates of the selected place

geolocator = Nominatim(user_agent="any_agent")
location = geolocator.geocode(place_box.value)

print(location.address)
print((location.latitude, location.longitude))

Hlavní město Praha, Praha, Česko
(50.0874654, 14.4212535)


In [26]:
# The folium package is used to plot our selected geolocation on a map

m = folium.Map(location=[location.latitude, location.longitude])
tooltip = location.latitude, location.longitude
folium.Marker([location.latitude, location.longitude], tooltip=tooltip).add_to(m)
display(m)

### Select a dataset

In [27]:
esm_file = home+"/data/CMIP6_ESM_colletion_file.json"
col = intake.open_esm_datastore(esm_file)

In [28]:
query = dict(source_id="CMCC-ESM2",
             experiment_id="ssp585",
             variable_id="tasmax",
             table_id="day"
)
cat = col.search(**query)
cat.df

,index,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,version,time_range,start_year,end_year,path
0,127629,ScenarioMIP,CMCC,CMCC-ESM2,ssp585,r1i1p1f1,day,tasmax,gn,v20210126,20150101-20391231,2015,2039,/home/jovyan/data/CMIP6/ScenarioMIP/CMCC/CMCC-...
1,127630,ScenarioMIP,CMCC,CMCC-ESM2,ssp585,r1i1p1f1,day,tasmax,gn,v20210126,20400101-20641231,2040,2064,/home/jovyan/data/CMIP6/ScenarioMIP/CMCC/CMCC-...
2,127631,ScenarioMIP,CMCC,CMCC-ESM2,ssp585,r1i1p1f1,day,tasmax,gn,v20210126,20650101-20891231,2065,2089,/home/jovyan/data/CMIP6/ScenarioMIP/CMCC/CMCC-...
3,127632,ScenarioMIP,CMCC,CMCC-ESM2,ssp585,r1i1p1f1,day,tasmax,gn,v20210126,20900101-21001231,2090,2100,/home/jovyan/data/CMIP6/ScenarioMIP/CMCC/CMCC-...


### Choose a time interval

In [29]:
start = widgets.SelectionSlider(
    options=range(int(min(cat.df["start_year"])),int(max(cat.df["end_year"]))+1),
    value=int(min(cat.df["start_year"])),
    description='Start Year:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
display(start)

SelectionSlider(continuous_update=False, description='Start Year:', options=(2015, 2016, 2017, 2018, 2019, 202…

In [30]:
end = widgets.SelectionSlider(
    options=range(start.value,int(max(cat.df["end_year"]))+1),
    value=start.value,
    description='End Year:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
display(end)

SelectionSlider(continuous_update=False, description='End Year:', options=(2015, 2016, 2017, 2018, 2019, 2020,…

### Open multiple files as a single dataset and subset over the chosen time interval

In [31]:
dset_dict = xr.open_mfdataset(cat.df["path"], parallel=True,engine="netcdf4")
dset_dict

<xarray.Dataset>
Dimensions:    (time: 31390, bnds: 2, lat: 192, lon: 288)
Coordinates:
  * time       (time) object 2015-01-01 12:00:00 ... 2100-12-31 12:00:00
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    height     float64 2.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object dask.array<chunksize=(9125, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(9125, 192, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(9125, 288, 2), meta=np.ndarray>
    tasmax     (time, lat, lon) float32 dask.array<chunksize=(9125, 192, 288), meta=np.ndarray>
Attributes: (12/48)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   60225.0
    branch_time_in_parent:  60225.0
    comment:                none
    ...                     ...
    title:                  CMCC-ESM2 output prepared for CMIP6
    variable_id:            tasmax
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by CMCC is licensed und...
    cmor_version:           3.6.0
    tracking_id:            hdl:21.14100/f0d0ff2e-4b19-4827-9359-9a85212ae1da

In [32]:
tas_year_xr = dset_dict["tasmax"].sel(time=dset_dict.time.dt.year.isin(range(start.value, end.value+1)))
tas_year_xr

<xarray.DataArray 'tasmax' (time: 13140, lat: 192, lon: 288)>
dask.array<getitem, shape=(13140, 192, 288), dtype=float32, chunksize=(9125, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2015-01-01 12:00:00 ... 2050-12-31 12:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    height   float64 2.0
Attributes:
    standard_name:  air_temperature
    long_name:      Daily Maximum Near-Surface Air Temperature
    comment:        maximum near-surface (usually, 2 meter) air temperature (...
    units:          K
    original_name:  TREFHTMX
    cell_methods:   area: mean time: maximum
    cell_measures:  area: areacella
    history:        2021-01-18T14:11:27Z altered by CMOR: Treated scalar dime...

### Find the nearest model coordinate

by finding the index of the nearest grid point

In [41]:
def find_nearest(lat_input, long_input):

    lat_index  = np.nanargmin((tas_year_xr["lat"]-lat_input)**2)
    lon_index = np.nanargmin((tas_year_xr["lon"]-long_input)**2)
    return lon_index,lat_index

([xloc, yloc]) = find_nearest(location.latitude,location.longitude) # xloc and yloc are the indices of the nearest model grid point

### Draw the map again

In [42]:
m = folium.Map(location=[location.latitude, location.longitude], zoom_start=8)

tooltip = location.latitude, location.longitude
folium.Marker(
    [location.latitude, location.longitude],
    tooltip=tooltip,
    popup="Location selected by You",
).add_to(m)


tooltip = float(tas_year_xr["lat"][yloc].values), float(tas_year_xr["lon"][xloc].values)
folium.Marker(
    [tas_year_xr["lat"][yloc], tas_year_xr["lon"][xloc]],
    tooltip=tooltip,
    popup="Model Grid Cell Center",
).add_to(m)


# Define coordinates of model grid cell (just for visualization)

rect_lat1_model = (tas_year_xr["lat"][yloc - 1] + tas_year_xr["lat"][yloc]) / 2
rect_lon1_model = (tas_year_xr["lon"][xloc - 1] + tas_year_xr["lon"][xloc]) / 2
rect_lat2_model = (tas_year_xr["lat"][yloc + 1] + tas_year_xr["lat"][yloc]) / 2
rect_lon2_model = (tas_year_xr["lon"][xloc + 1] + tas_year_xr["lon"][xloc]) / 2


# Draw model grid cell

folium.Rectangle(
    bounds=[[rect_lat1_model, rect_lon1_model], [rect_lat2_model, rect_lon2_model]],
    color="#ff7800",
    fill=True,
    fill_color="#ffff00",
    fill_opacity=0.2,
).add_to(m)

m

### Get data for the nearest model coordinate and plot it on a interactive graph
 - Select the chosen spatial point
 - Convert Kelvin to °C
 - Convert `CFTimeIndex` to a Pandas `DatetimeIndex` to avoid plotting issues (labels, indexes, ...) 
 - Create a dataframe to accommodate a) the selected model data and b) the Summer Day Threshold
 - Plot data with `hvplot`

In [43]:
tasmax_year_place_xr = tas_year_xr[:, yloc, xloc] - 273.15
tasmax_year_place_xr["time"] = tasmax_year_place_xr.indexes["time"].to_datetimeindex()
tasmax_year_place_xr

<xarray.DataArray 'tasmax' (time: 13140)>
dask.array<sub, shape=(13140,), dtype=float32, chunksize=(9125,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2015-01-01T12:00:00 ... 2050-12-31T12:00:00
    lat      float64 50.42
    lon      float64 15.0
    height   float64 2.0

In [44]:
tasmax_year_place_df = pd.DataFrame(index = tasmax_year_place_xr['time'].values, 
                                    columns = ['Model Temperature', 'Summer Day Threshold'])

tasmax_year_place_df.loc[:, 'Model Temperature'] = tasmax_year_place_xr.values # Model data
tasmax_year_place_df.loc[:, 'Summer Day Threshold'] = 25                       # Summer Day Threshold

# Plot data and define title and legend
tasmax_year_place_df.hvplot.line(y=['Model Temperature', 'Summer Day Threshold'], 
                                 value_label='Temperature in °C', legend='bottom', 
                                 title='Daily-Maximum Near-Surface Air Temperature for '+place_box.value, 
                                 height=500, width=620)

:NdOverlay   [Variable]
   :Curve   [index]   (Temperature in °C)

### Compute the Summer days climate index

Starting from the daily maximum temperature `TX`,
the **Summer Days** index is the number of days where $TX > T$ (`T` is  a reference temperature, e.g. 25°C)

In [45]:
summer_days = xr.where(tasmax_year_place_xr>25.0, 1, 0).groupby('time.year').sum()
summer_days

<xarray.DataArray 'tasmax' (year: 36)>
dask.array<stack, shape=(36,), dtype=int64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
    lat      float64 50.42
    lon      float64 15.0
    height   float64 2.0
  * year     (year) int64 2015 2016 2017 2018 2019 ... 2046 2047 2048 2049 2050

In [46]:
summer_days_df = pd.DataFrame(index = summer_days['year'].values, 
            columns = ['Summer Days'])

summer_days_df.loc[:, 'Summer Days'] = summer_days.values

summer_days_df.hvplot.line(y=['Summer Days'],
            value_label='Summer Days', legend='bottom', 
            title='Number of Summer Days in '+place_box.value+" in "+str(start.value)+"-"+str(end.value), 
            height=500, width=620)

:Curve   [index]   (Summer Days)